In [9]:
from tensorflow.keras.layers import Embedding, LSTM


In [17]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

nltk.download('punkt')

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('health.json').read()
intents = json.loads(data_file)

lemmatizer = WordNetLemmatizer()

for intent in intents['diseases']:
    for pattern in intent['symptoms']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['name']))
        if intent['name'] not in classes:
            classes.append(intent['name'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

pickle.dump(words, open('words_rnn.pkl', 'wb'))
pickle.dump(classes, open('classes_rnn.pkl', 'wb'))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\parik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
training_rnn = []
output_empty_rnn = [0] * len(classes)

for doc in documents:
    bag_rnn = []
    pattern_words_rnn = doc[0]
    pattern_words_rnn = [lemmatizer.lemmatize(word.lower()) for word in pattern_words_rnn]
    for w in words:
        bag_rnn.append(1) if w in pattern_words_rnn else bag_rnn.append(0)

    output_row_rnn = list(output_empty_rnn)
    output_row_rnn[classes.index(doc[1])] = 1

    training_rnn.append([np.array(bag_rnn), np.array(output_row_rnn)])

train_x_rnn = [item[0] for item in training_rnn]
train_y_rnn = [item[1] for item in training_rnn]

train_x_rnn = pad_sequences(train_x_rnn)  # Apply pad_sequences here
train_y_rnn = np.array(train_y_rnn)

model_rnn = Sequential()
model_rnn.add(Embedding(input_dim=len(words), output_dim=128, input_length=train_x_rnn.shape[1]))
model_rnn.add(LSTM(128))
model_rnn.add(Dropout(0.5))
model_rnn.add(Dense(len(classes), activation='softmax'))

model_rnn.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

hist_rnn = model_rnn.fit(train_x_rnn, train_y_rnn, epochs=200, batch_size=5, verbose=1)
final_training_accuracy_rnn = hist_rnn.history['accuracy'][-1]
print(f'RNN Model - Final Training Accuracy: {final_training_accuracy_rnn * 100:.2f}%')

plt.plot(hist_rnn.history['accuracy'])
plt.title('RNN Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

model_rnn.save('chatbot_model_rnn.h5', hist_rnn)
print("RNN Model created")

Epoch 1/200
313/313 [==============================] - 72s 224ms/step - loss: 5.6149 - accuracy: 0.0019
Epoch 2/200
313/313 [==============================] - 72s 229ms/step - loss: 5.5933 - accuracy: 0.0090
Epoch 3/200
313/313 [==============================] - 89s 284ms/step - loss: 5.5789 - accuracy: 0.0083
Epoch 4/200
313/313 [==============================] - 97s 310ms/step - loss: 5.5705 - accuracy: 0.0096
Epoch 5/200
313/313 [==============================] - 94s 299ms/step - loss: 5.5631 - accuracy: 0.0115
Epoch 6/200
313/313 [==============================] - 76s 243ms/step - loss: 5.5405 - accuracy: 0.0135
Epoch 7/200
313/313 [==============================] - 78s 249ms/step - loss: 5.5230 - accuracy: 0.0147
Epoch 8/200
313/313 [==============================] - 78s 249ms/step - loss: 5.5303 - accuracy: 0.0122
Epoch 9/200
313/313 [==============================] - 76s 244ms/step - loss: 5.5212 - accuracy: 0.0186
Epoch 10/200
313/313 [==============================] - 78s 250m